In [1]:
from metatrader.utils.factory import MetatraderFactory
metatrader = MetatraderFactory.get_metatrader()
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import pytz

mt5 = metatrader.connect()

symbol = 'SP500'
timeframe = mt5.TIMEFRAME_M5

trade_volume = 10
commission = 0.25



Connected to Metatrader successfully.


In [2]:
bars = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, 0,200))
bars['time'] = pd.to_datetime(bars['time'], unit='s')
bars['trade_volume'] = trade_volume
bars

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume
0,2023-10-24 07:20:00,4228.6,4229.1,4228.1,4228.6,96,2,0,10
1,2023-10-24 07:25:00,4228.3,4228.3,4227.1,4227.3,42,3,0,10
2,2023-10-24 07:30:00,4227.1,4227.8,4226.9,4227.8,36,2,0,10
3,2023-10-24 07:35:00,4227.5,4228.3,4227.5,4228.3,47,2,0,10
4,2023-10-24 07:40:00,4228.4,4228.6,4228.3,4228.6,39,3,0,10
...,...,...,...,...,...,...,...,...,...
195,2023-10-24 23:35:00,4249.7,4249.9,4248.1,4249.4,174,2,0,10
196,2023-10-24 23:40:00,4249.1,4250.6,4248.6,4249.1,90,1,0,10
197,2023-10-24 23:45:00,4249.0,4249.1,4245.6,4247.4,94,1,0,10
198,2023-10-24 23:50:00,4247.5,4247.6,4245.5,4246.0,141,2,0,10


In [3]:
bars['ema_20'] = bars['close'].ewm(span=20).mean()
bars['ema_10'] = bars['close'].ewm(span=10).mean()
bars['hour'] = bars['time'].dt.hour
bars

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,ema_20,ema_10,hour
0,2023-10-24 07:20:00,4228.6,4229.1,4228.1,4228.6,96,2,0,10,4228.600000,4228.600000,7
1,2023-10-24 07:25:00,4228.3,4228.3,4227.1,4227.3,42,3,0,10,4227.917500,4227.885000,7
2,2023-10-24 07:30:00,4227.1,4227.8,4226.9,4227.8,36,2,0,10,4227.874355,4227.850831,7
3,2023-10-24 07:35:00,4227.5,4228.3,4227.5,4228.3,47,2,0,10,4227.997232,4227.998812,7
4,2023-10-24 07:40:00,4228.4,4228.6,4228.3,4228.6,39,3,0,10,4228.143036,4228.171397,7
...,...,...,...,...,...,...,...,...,...,...,...,...
195,2023-10-24 23:35:00,4249.7,4249.9,4248.1,4249.4,174,2,0,10,4248.783438,4249.887413,23
196,2023-10-24 23:40:00,4249.1,4250.6,4248.6,4249.1,90,1,0,10,4248.813587,4249.744247,23
197,2023-10-24 23:45:00,4249.0,4249.1,4245.6,4247.4,94,1,0,10,4248.678959,4249.318020,23
198,2023-10-24 23:50:00,4247.5,4247.6,4245.5,4246.0,141,2,0,10,4248.423820,4248.714744,23


In [4]:
bars2 = bars.dropna().copy()

def get_signal(x):

    if 9 <= x['hour'] <= 22:

        if x['ema_10'] > x['ema_20']:
            x["position"] = "long"
            return 1
        elif x['ema_10'] < x['ema_20']:
            x["position"] = "short"
            return -1

    return 0

bars2['signal'] = bars2.apply(get_signal, axis=1)
bars2['prev_price'] = bars2['close'].shift(1)
bars2['price_change'] = bars2['close'] - bars2['prev_price']

bars2['signal_change'] = (bars2['signal'] - bars2['signal'].shift(1)).abs()
bars2['commission'] = bars2.apply(lambda x: commission * x['signal_change'] * x['trade_volume'], axis=1)
bars2

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,ema_20,ema_10,hour,signal,prev_price,price_change,signal_change,commission
0,2023-10-24 07:20:00,4228.6,4229.1,4228.1,4228.6,96,2,0,10,4228.600000,4228.600000,7,0,NaN,NaN,NaN,NaN
1,2023-10-24 07:25:00,4228.3,4228.3,4227.1,4227.3,42,3,0,10,4227.917500,4227.885000,7,0,4228.6,-1.3,0.0,0.0
2,2023-10-24 07:30:00,4227.1,4227.8,4226.9,4227.8,36,2,0,10,4227.874355,4227.850831,7,0,4227.3,0.5,0.0,0.0
3,2023-10-24 07:35:00,4227.5,4228.3,4227.5,4228.3,47,2,0,10,4227.997232,4227.998812,7,0,4227.8,0.5,0.0,0.0
4,2023-10-24 07:40:00,4228.4,4228.6,4228.3,4228.6,39,3,0,10,4228.143036,4228.171397,7,0,4228.3,0.3,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2023-10-24 23:35:00,4249.7,4249.9,4248.1,4249.4,174,2,0,10,4248.783438,4249.887413,23,0,4249.6,-0.2,0.0,0.0
196,2023-10-24 23:40:00,4249.1,4250.6,4248.6,4249.1,90,1,0,10,4248.813587,4249.744247,23,0,4249.4,-0.3,0.0,0.0
197,2023-10-24 23:45:00,4249.0,4249.1,4245.6,4247.4,94,1,0,10,4248.678959,4249.318020,23,0,4249.1,-1.7,0.0,0.0
198,2023-10-24 23:50:00,4247.5,4247.6,4245.5,4246.0,141,2,0,10,4248.423820,4248.714744,23,0,4247.4,-1.4,0.0,0.0


In [5]:
bars3 = bars2.dropna().copy()
bars3['profit'] = bars3['signal'] * bars3['price_change'] * bars3['trade_volume']
bars3['gross_profit'] = bars3['profit'].cumsum()
bars3['net_profit'] = bars3['gross_profit'] - bars3['commission'].cumsum()


bars3

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,ema_20,ema_10,hour,signal,prev_price,price_change,signal_change,commission,profit,gross_profit,net_profit
1,2023-10-24 07:25:00,4228.3,4228.3,4227.1,4227.3,42,3,0,10,4227.917500,4227.885000,7,0,4228.6,-1.3,0.0,0.0,-0.0,-0.0,-0.0
2,2023-10-24 07:30:00,4227.1,4227.8,4226.9,4227.8,36,2,0,10,4227.874355,4227.850831,7,0,4227.3,0.5,0.0,0.0,0.0,0.0,0.0
3,2023-10-24 07:35:00,4227.5,4228.3,4227.5,4228.3,47,2,0,10,4227.997232,4227.998812,7,0,4227.8,0.5,0.0,0.0,0.0,0.0,0.0
4,2023-10-24 07:40:00,4228.4,4228.6,4228.3,4228.6,39,3,0,10,4228.143036,4228.171397,7,0,4228.3,0.3,0.0,0.0,0.0,0.0,0.0
5,2023-10-24 07:45:00,4228.7,4229.3,4228.3,4229.1,32,3,0,10,4228.344912,4228.412587,7,0,4228.6,0.5,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2023-10-24 23:35:00,4249.7,4249.9,4248.1,4249.4,174,2,0,10,4248.783438,4249.887413,23,0,4249.6,-0.2,0.0,0.0,-0.0,344.0,309.0
196,2023-10-24 23:40:00,4249.1,4250.6,4248.6,4249.1,90,1,0,10,4248.813587,4249.744247,23,0,4249.4,-0.3,0.0,0.0,-0.0,344.0,309.0
197,2023-10-24 23:45:00,4249.0,4249.1,4245.6,4247.4,94,1,0,10,4248.678959,4249.318020,23,0,4249.1,-1.7,0.0,0.0,-0.0,344.0,309.0
198,2023-10-24 23:50:00,4247.5,4247.6,4245.5,4246.0,141,2,0,10,4248.423820,4248.714744,23,0,4247.4,-1.4,0.0,0.0,-0.0,344.0,309.0


In [6]:
px.line(bars3, 'time', ['gross_profit', 'net_profit'], title="SP500 NY session Backtest EMA Trendfollowing")


c:\Users\praveenjaisankar\AppData\Local\pypoetry\Cache\virtualenvs\metatrader-Cdlomroj-py3.10\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [7]:

trades = bars3.loc[bars3['signal_change']!=0]


trades


,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,ema_20,ema_10,hour,signal,prev_price,price_change,signal_change,commission,profit,gross_profit,net_profit
20,2023-10-24 09:00:00,4230.5,4232.4,4230.5,4232.1,150,2,0,10,4230.291708,4230.636129,9,1,4230.8,1.3,1.0,2.5,13.0,1.300000e+01,10.5
40,2023-10-24 10:40:00,4229.5,4230.8,4227.5,4228.6,365,1,0,10,4232.790324,4232.708124,10,-1,4229.7,-1.1,2.0,5.0,11.0,-9.094947e-12,-7.5
55,2023-10-24 11:55:00,4232.8,4234.5,4232.0,4233.4,132,2,0,10,4230.854054,4230.887079,11,1,4232.8,0.6,2.0,5.0,6.0,-3.600000e+01,-48.5
107,2023-10-24 16:15:00,4240.5,4240.8,4236.5,4236.8,159,1,0,10,4240.896259,4240.594640,16,-1,4240.8,-4.0,2.0,5.0,40.0,7.800000e+01,60.5
110,2023-10-24 16:30:00,4238.5,4248.3,4238.0,4248.2,1086,1,0,10,4241.103365,4241.259423,16,1,4238.3,9.9,2.0,5.0,99.0,1.620000e+02,139.5
133,2023-10-24 18:25:00,4242.7,4245.0,4240.8,4241.4,764,1,0,10,4248.712511,4248.406719,18,-1,4242.7,-1.3,2.0,5.0,13.0,1.200000e+02,92.5
164,2023-10-24 21:00:00,4233.2,4235.9,4231.7,4233.1,523,1,0,10,4232.400478,4232.446747,21,1,4233.3,-0.2,2.0,5.0,-2.0,1.990000e+02,166.5
188,2023-10-24 23:00:00,4247.9,4267.0,4244.6,4250.1,1546,1,0,10,4247.287337,4249.493771,23,0,4247.6,2.5,1.0,2.5,0.0,3.440000e+02,309.0


In [8]:
fig = go.Figure(data=[go.Candlestick(x=trades['time'],
                open=trades['open'],
                high=trades['high'],
                low=trades['low'],
                close=trades['close'])],
                )

fig.show()

c:\Users\praveenjaisankar\AppData\Local\pypoetry\Cache\virtualenvs\metatrader-Cdlomroj-py3.10\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

